<i><b>Public AI</b></i>
<br>
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 1. DNN Basis**
# Section 7. 역전파 구현하기

### _Objective_
1. **계산그래프란?**:역전파 알고리즘을 이해하기 위해 활용되는 계산 그래프 표현 방식을 배웁니다. 
2. **계산 그래프로 기울기 계산하기**: 계산 그래프를 활용해 단계별로 역전파 알고리즘을 배웁니다.<br>


# \[ 1. 계산그래프란? \]
 **계산 그래프**란, 텐서플로우의 그래프(graph)와 같이 Node와 Edge로 수식을 정의하는 방법을 의미합니다.

## 1. 계산그래프로 수식 정의 하기

아래와 같이 단순한 수식을 계산그래프로 나타내봅시다.

$$
f(x,y,z) = (x + y)*z
$$

### (1) 수식을 그래프로 나타내기

<img src="https://i.imgur.com/5BcvlA3.png" width="800" height="600"/>

각각의 Node는 입력받은 값에 수행할 연산을 기록하며, Node의 좌측 Edge에서는 입력되는 값이, 우측 Edge로는 연산 결과값이 흘러가게 됩니다. 

### (2) 계산 그래프로 순전파(FeedForward) 적용하기
이제 계산 그래프를 이용해 왼쪽부터 오른쪽까지 순차적으로 계산하는 순전파 계산 결과를 구해봅시다. 

$ x=-2, y=5, z=-4 $일 때, $f(x,y,z)$의 결과를 구해주세요.

<img src="https://i.imgur.com/tT7d1wr.png" width="800" height="600"/>

위와 같이 순서 대로 하나씩 계산을 적용하면 됩니다.

# \[ 2. 계산 그래프로 기울기 계산하기 \]

앞서 왼쪽부터 오른쪽까지 순전파로 덧셈과 곱셈이 진행되는 과정을 살펴보았습니다. 그렇다면 역전파도 계산 그래프를 이용해 표현할 수 있을까요? 계산 그래프로 역전파 연산을 하기 위해서는 몇 가지 간단한 미분 법칙을 알아야 합니다. 가장 기본이 되는 수식을 위주로 파악하겠습니다.

## 1. 덧셈/곱셈/행렬곱 역전파

### (1) 덧셈 노드 역전파

덧셈 노드의 역전파란, 덧셈의 결과로 얻은 값이 아주 조금 변할 때, 입력했던 데이터의 값은 얼마나 변할지 구하는 연산 과정을 뜻합니다. 아래 왼쪽 이미지를 예로 들어 살펴봅시다. 왼쪽 계산그래프에 나타난 덧셈 연산을 역전파 한다는 것은, 결과값 $z$가 아주 조금 변할 때, 입력값 $x$가 얼마나 변할지 연산해본다는 뜻입니다. 즉, 덧셈을 미분해본다는 것이죠.

더하기의 미분은 아래와 같습니다.

$$
z = x + y \mbox{일 때,}\\
\frac{\partial z}{\partial x} = 1 \\
\frac{\partial z}{\partial y} = 1 \\
$$

위 수식을 계산그래프로 표현하면 아래와 같습니다. 

왼쪽 이미지는 $z = x + y$를 계산 그래프로 나타낸 것입니다. 덧셈 노드를 통과한 $z$는 그 자체로 최종 결과값이 될 수도 있고, 이후 더 많은 노드를 통과하여 결과값을 반환 할 수도 있습니다. 모든 노드를 통과한 후 얻게 된 예측값과 정답값 사이의 차이(Loss, $L$)를 최소화 하도록 역전파 하는 이미지입니다. 오른쪽 이미지를 보면 결국 덧셈 노드의 역전파는 1을 곱할 뿐, 입력한 값을 그대로 다음 노드로 전달하게 됨을 알 수 있습니다.

<img src="https://i.imgur.com/f3xJ5LO.png" width="800" height="600"/>

### (2) 곱셈 노드 역전파

곱하기의 미분은 아래와 같습니다.

$$
z = x * y \mbox{일 때,}\\
\frac{\partial z}{\partial x} = y \\
\frac{\partial z}{\partial y} = x \\
$$

위를 계산그래프로 표현하면 아래와 같습니다.

<img src="https://i.imgur.com/dXCZ7AA.png" width="800" height="600"/>

### (3) 행렬곱 노드 역전파

딥러닝에서 연산의 다수를 차지하는 것은 앞서 살펴본 덧셈이나 곱셈이 아닌, 행렬곱 연산입니다. 행렬곱은 $*$와 $+$로 이루어져 있습니다.

행렬곱의 미분은 아래와 같습니다.
$$
Z = X \cdot W \mbox{일 때,}\\
\frac{\partial Z}{\partial X} = W^T \\
\frac{\partial Z}{\partial W} = X^T \\
$$

여기서 주의해야 하는 것은 전치관계입니다. 행과 열이 바뀌어야만 합니다.

<img src="https://i.imgur.com/5qdspwt.png" width="800" height="600"/><br>

여기서 행렬이 전치되어야 하는 이유에 대해 알아보겠습니다. 예시 행렬의 shape를 보면 전치되어야 하는 이유를 알 수 있습니다.<br>
예시 행렬의 shape이 아래와 같을 때,

![](https://i.imgur.com/cMkPoRJ.png)

계산그래프에 shape를 표시하면 다음과 같습니다.

![](https://i.imgur.com/ZFae6Sw.png)

scalar를 $(m,n)$ 행렬로 미분했을 때의 결과는 아래와 같은데,

<img src="https://i.imgur.com/PYCaLNc.png" width="300" height="225"/><br>

scalar를 행렬로 미분했을 때의 shape은 미분한 행렬의 shape과 같다는 것을 확인할 수 있습니다. 이를 통해 $\frac{\partial L}{\partial W}$와 $\frac{\partial L}{\partial X}$ 의 shape을 알 수 있습니다.

<img src="https://i.imgur.com/vP1lDuI.png" width="300" height="225"/><br>

행렬곱을 할 때 앞 행렬의 열의 수와 뒤 행렬의 행의 수가 같아야 한다는 행렬곱의 조건에 의해 $\frac { \partial L }{ \partial Z } \ast W$의 행렬곱은 성립되지 않으므로 $W$를 transpose(전치)해주어야 하는 것입니다. <br><br>
$W$대신 $W$의 전치행렬 $W^T$을 사용하면 행렬곱이 성립합니다.

<img src="https://i.imgur.com/6gzuBWn.png" width="300" height="225"/><br>

## 2. 연쇄 법칙과 역전파

### (1) 연쇄 법칙 계산 그래프로 표현하기 
앞서 Section 4에서 합성함수의 도함수가 각 도함수 끼리의 곱의 형태로 이루어지는 것을 연쇄법칙(chain rule)이라고 한다고 배웠습니다. 연쇄 법칙을 일반화해서 계산 그래프에서 본다면, 아래와 같습니다. 

![Imgur](https://i.imgur.com/lotK0Ag.png)

역방향으로 연쇄법칙을 적용한다면 아래와 같습니다.

![Imgur](https://i.imgur.com/P8E5eYg.png)

### (2) 연쇄 법칙으로 수식 풀어보기

이전 Section에서 다루었던 예제인 $z = (x+y)^2$라는 수식을 연쇄법칙으로 풀어보도록 하겠습니다.

$$
t = x+y \\
z = t^2
$$

위와 같이 두개의 함수로 나눌 수 있습니다. 여기서 연쇄법칙을 적용하면

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}*\frac{\partial t}{\partial x}
$$

각각의 편미분($\frac{\partial z}{\partial t}$,$\frac{\partial t}{\partial x}$)은 아래와 같이 구할 수 있습니다.

$$
\frac{\partial z}{\partial t} = 2t, \\
\frac{\partial t}{\partial x} = 1,
$$

위에서 구한 편미분을 통해 우리는 아래와 같이 구할 수 있습니다.

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}*\frac{\partial t}{\partial x} = 2t * 1 = 2(x+y)
$$

### (3) 계산 그래프로 수식 풀어보기

아래 계산그래프는 위에서 다루었던 수식을 계산그래프로 나타낸 것입니다. 위의 『(2) 연쇄 법칙으로 수식 풀어보기』와 비교하며 보면 이해가 쉽습니다.

![](https://i.imgur.com/C04Pzi8.png)

연쇄법칙에 따라 하나씩 풀어 쓰면 아래와 같습니다.

![](https://i.imgur.com/xfuWFnF.png)

각 부분의 편미분 수식을 적용하면 아래와 같습니다.


![](https://i.imgur.com/jNfHwx3.png)

$z$ 값이 1 만큼 변화하면, $t$ 값은  $1*2t$ 만큼 변화하고, 최종적으로 $x$는 $2(x+y)$만큼 변화한다는 의미입니다. 

## 3. 활성화 함수 역전파

### (1) Sigmoid 역전파

시그모이드 함수 차례입니다. 시그모이드 함수는 아래와 같습니다.

$$
y = \frac{1}{1+exp(-x)}
$$

계산그래프로 구성한다면 아래와 같습니다.

![](https://i.imgur.com/cfhdqNy.png)

시그모이드 함수의 결과로 얻어진 $y$ 값이 아주 조금 변할 때, $x$ 값은 얼마나 변하게 될까요? 시그모이드 함수의 계산그래프를 이용해여 역전파를 한 단계씩 순서대로 진행해봅시다. 

#### 1 단계
계산그래프의 흐름과 연쇄법칙에 따라 나눗셈의 미분이 곱해집니다.
![](https://i.imgur.com/vXy8ohM.png)

#### 2 단계
연쇄법칙에 따라 덧셈노드를 통과하며 1이 곱해집니다. 즉, 받은 입력 값을 그대로 앞의 노드로 전달합니다. 

![](https://i.imgur.com/Wp2vUeY.png)

#### 3 단계
연쇄법칙에 따라 지수함수를 통과하며 지수함수의 미분이 곱해집니다.
![](https://i.imgur.com/4C8msYE.png)

#### 4 단계
연쇄법칙에 따라 곱셈노드를 통과하며 $-x$의 미분값인 $-1$이 곱해집니다.
![](https://i.imgur.com/EjuUNat.png)

#### 수식 정리

시그모이드 함수의 역전파를 계산 그래프로 그려 한 단계씩 살펴보았습니다. 결과로 얻어진 $\frac{\partial L}{\partial y} * (-y^2) * e^{(-x)} * (-1)$ 를 정리하면 아래와 같습니다.

$$
\frac{\partial L}{\partial y} * (-y^2) * e^{(-x)} * (-1) = \frac{\partial L}{\partial y} * y^2 * e^{(-x)}\\
= \frac{\partial L}{\partial y}\frac{1}{(1+e^{-x})^2}*e^{(-x)}\\
= \frac{\partial L}{\partial y}\frac{1}{(1+e^{-x})}*\frac{e^{(-x)}}{1+e^{(-x)}}\\
= \frac{\partial L}{\partial y}\frac{1}{(1+e^{-x})}*(1 - \frac{1}{1+e^{(-x)}})\\
= \frac{\partial L}{\partial y}y(1-y)
$$

### (2) Relu 역전파

ReLU는 앞서 몇 번 등장했는데, 활성화 함수로 현재 가장 많이 쓰이는 함수 중 하나입니다.<br>

$$
relu(x) = \begin{cases}
x, & \mbox{if x > 0} \\
0, & \mbox{if x <= 0}
\end{cases}
$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

relu = lambda x : np.maximum(x,0)
xs = np.linspace(-1,1,11)
ys = relu(xs)

plt.title('RELU function')
plt.plot(xs,ys)
plt.show()

<Figure size 640x480 with 1 Axes>

$$
\frac{\partial relu}{\partial x} = \begin{cases}
1, & \mbox{if x > 0} \\
0, & \mbox{if x <= 0}
\end{cases}
$$

$x$가 0보다 큰 경우와 $x$가 0보다 작거나 같은 경우에 각각 다른 수식이 있기 때문에, 계산그래프로 구성한다면 아래와 같이 두 개의 계산 그래프로 표현할 수 있습니다.

![](https://i.imgur.com/I4eBaJT.png)

$x$가 0보다 큰 경우, $x$ 값이 그대로 $y$값이 되는 것이 ReLU입니다. 따라서 이 경우 역전파 연산을 하게 되면 출력값 방향에서(이 경우 우측 노드)에서 받은 미분 값이 그대로 입력값 방향(이 경우 왼쪽 노드)로 전달되게 됩니다. 한편, $x$가 0보다 작거나 같은 경우에는 0을 반환하게 되는데, 이 경우에는 역전파 연산을 하게 되면 출력값 방향에서 받은 미분 값에 상관 없이 0을 입력값 방향으로 전달하게 됩니다.


## 4. 손실 함수 역전파

### (1) Sigmoid-with-Loss 역전파

시그모이드 함수와 binary-Cross-Entropy는 딥러닝에서 일반적으로 함께 쓰이는 손실함수입니다.<br>

$$
\mbox{sigmoid 함수 : }y = \frac{1}{1+exp(-x)} \\
\mbox{Cross Entropy 함수 : } Loss = -(y_{true}log(y_{pred}) + (1-y_{true})(log(1-y_{pred}))
$$

계산 그래프로 표현하면 아래와 같습니다.

![](https://i.imgur.com/BtI1Srd.png)

수식으로 미분과정을 계산하면 아래와 같습니다.

$
\frac{\partial L}{\partial z} = \frac{\partial L}{\partial a}\frac{\partial a}{\partial z} \\
= \frac{\partial}{\partial a}\{-ylog(a)+(1-y)log(1-a)\}\frac{\partial a}{\partial z}\\
= \{\frac{-y}{a}+(1-y)\frac{-1}{1-a}\}\frac{\partial a}{\partial z}\\
= \{\frac{-(1-a)y+(1-y)a}{(1-a)a} \}\frac{\partial a}{\partial z}\\
= \frac{(a-y)}{(1-a)a}\frac{\partial a}{\partial z}\\
= a-y\\
\therefore \frac{\partial L}{\partial z} = a-y
$

![](https://i.imgur.com/3no6oJi.png)

### (2) Mean-Squared-Error 역전파

Linear Regression 등 회귀 문제에서는 주로 MSE(Mean-Squared-Error) 손실함수를 많이 이용합니다. 보통 MSE를 손실함수로 이용할 때, 앞에 $\frac{1}{2}$을 붙이는데, 이는 미분하였을 때 발생하는 2라는 상수를 상쇄시키기 위함입니다.

$$
MSE  = \frac{1}{2}(y_{pred}-y_{true})^2
$$


미분 식은 아래와 같습니다.

$$
\frac{\partial MSE}{\partial y_{pred}} = {y_{pred}-y_{true}}\\
$$

<img src="https://i.imgur.com/FG6xGwY.png" width="600" height="450"/>

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/03/04


---